# Using Huggingface

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [1]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_MiniLM-L6')


ModuleNotFoundError: No module named 'sentence_transformers'

### Supporting functions

In [31]:
import PyPDF2
def read_jd_file(filename):
    """Program to read the entire file (absolute path) using read() function"""
    file = open(filename, "r")
    content = file.read()
    file.close()
    return content

def read_resume_text(pdffile):
    """Read PDF file ans=d parse as text"""
    pdf = PyPDF2.PdfReader(pdffile)
    resume = ""

    for i in range(len(pdf.pages)):
        pageObj = pdf.pages[i]
        resume += pageObj.extract_text()
    return resume

## Comparing the similarity between the JDs

In [26]:
jd_1 = read_jd_file("./../CV-Job-matching-main/JD/jd_database analyst_1.txt")
jd_2 = read_jd_file("./../CV-Job-matching-main/JD/jd_Oracle DBA_2.txt")
jd_3 = read_jd_file("./../CV-Job-matching-main/JD/jd_Oracle DBA_3.txt")

sentences = [jd_1,jd_2,jd_3]
# into vector
embeddings = model.encode(sentences)
# compute cosine similarity
cos_sim = util.cos_sim(embeddings, embeddings)

In [21]:
# add all pairs to a list with their cosim similarity
all_sentence_combination = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combination.append([cos_sim[i][j],i,j])

# sort list
all_sentence_combination = sorted(all_sentence_combination, key=lambda x: x[0], reverse=True)

print('Sentence pair similar range: ')
for score, i, j in all_sentence_combination:
    print(f'Comparing JD {i} with JD {j} get Score: {format(cos_sim[i][j],".0%")}')

Sentence pair similar range: 
Comparing JD 1 with JD 2 get Score: 69%
Comparing JD 0 with JD 2 get Score: 53%
Comparing JD 0 with JD 1 get Score: 42%


## Comparing the CV with each of the JD

In [32]:
jd_1 = read_jd_file("./../CV-Job-matching-main/JD/jd_database analyst_1.txt")
jd_2 = read_jd_file("./../CV-Job-matching-main/JD/jd_Oracle DBA_2.txt")
jd_3 = read_jd_file("./../CV-Job-matching-main/JD/jd_Oracle DBA_3.txt")

cv1 = read_resume_text("./../CV-Job-matching-main/CV/Akshay_Srimatrix.pdf")
sentences = [cv1, jd_1,jd_2,jd_3]
# into vector
embeddings = model.encode(sentences)
# compute cosine similarity
cos_sim = util.cos_sim(embeddings, embeddings)

In [36]:
# add all pairs to a list with their cosim similarity
all_sentence_combination = []
for i in range(len(cos_sim)):
    all_sentence_combination.append([cos_sim[0][i],i])

# sort list
all_sentence_combination = sorted(all_sentence_combination, key=lambda x: x[0], reverse=True)

print('Sentence pair similar range: ')
for score, i in all_sentence_combination:
    if (i>0):
        print(f'Comparing CV with JD {i-1} get Score: {format(cos_sim[0][i],".0%")}')

Sentence pair similar range: 
Comparing CV with JD 1 get Score: 63%
Comparing CV with JD 2 get Score: 54%
Comparing CV with JD 0 get Score: 33%


In [ ]:
def matching_score_visualization(similarity):
    fig = go.Figure(go.Indicator(
        domain = {'x': [0, 1], 'y': [0, 1]},
        value = similarity,
        mode = "gauge+number",
        title = {'text': "Matching percentage (%)"},
        #delta = {'reference': 100},
        gauge = {
            'axis': {'range': [0, 100]},
            'steps' : [
                {'range': [0, 50], 'color': "#FFB6C1"},
                {'range': [50, 70], 'color': "#FFFFE0"},
                {'range': [70, 100], 'color': "#90EE90"}
            ],
                 'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))
    
    fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
    fig.show()
    # Print notification
    if similarity < 50:
        print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
    elif similarity >= 50 and similarity < 70:
        print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
    else:
        print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))

for score, i in all_sentence_combination:
    if (i>0):
        print(f'Comparing CV with JD {i-1} get Score: {format(cos_sim[0][i],".0%")}')
        matching_score_visualization(format(cos_sim[0][i],".0%"))